<a href="https://colab.research.google.com/github/arguntolga/ts_jupyter/blob/main/superstock1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd

# Define your two stock tickers
#stock_list = ["EREGL.IS", "FROTO.IS", "THYAO.IS", "TTRAK.IS", "MIATK.IS"]
stock_list = [
    "AKBNK.IS", "ARCLK.IS", "ASELS.IS", "BIMAS.IS", "BRISA.IS", "ASTOR.IS"
    #"DOHOL.IS", "SASA.IS", "EKGYO.IS", "ENKAI.IS", "EREGL.IS", "FROTO.IS",
    #"GARAN.IS", "GUBRF.IS", "HALKB.IS", "HEKTS.IS", "ISCTR.IS", "KCHOL.IS",
    #"KOZAA.IS", "KOZAL.IS", "KRDMD.IS", "MGROS.IS", "TTRAK.IS", "PETKM.IS",
    #"PGSUS.IS", "SAHOL.IS", "SISE.IS", "TAVHL.IS", "THYAO.IS", "TKFEN.IS",
    #"TTKOM.IS", "TUPRS.IS", "ULKER.IS", "VESTL.IS", "YKBNK.IS", "VESBE.IS"
]

# Set start and end dates
start_date = "2023-01-04"
end_date = "2024-05-24"

# Download data using yfinance
data = yf.download(stock_list, start=start_date, end=end_date)["Adj Close"]

print(data)

def calculate_moving_averages(stock_list, data, window_sizes):
  for stock in stock_list:
    for window in window_sizes:
      data[stock + '_MA_' + str(window)] = data[stock].rolling(window=window).mean()
  return data

def calculate_above_below_ma(stock_list, data, window_sizes):
  above_below_counts = {}
  for date in data.index:
    daily_counts = {"Date": date}
    for window in window_sizes:
      above_count = 0
      for stock in stock_list:
        ma_col = f"{stock}_MA_{window}"
        above_count += (data.loc[date, stock] > data.loc[date, ma_col])
      below_count = len(stock_list) - above_count
      daily_counts["Above MA" + str(window)] = above_count
      daily_counts["Below MA" + str(window)] = below_count
    above_below_counts[date] = daily_counts
  return pd.DataFrame(above_below_counts).transpose()

def calculate_super_stock(stock_list, data, window_sizes):
  for date in data.index:
    for stock in stock_list:
      super_stock = 1
      for window in window_sizes:
        ma_col = f"{stock}_MA_{window}"
        if data.loc[date, stock] < data.loc[date, ma_col]:
          super_stock = 0
        for next_window_idx in range(1, len(window_sizes)):
          next_ma_col = f"{stock}_MA_{window_sizes[next_window_idx]}"
          if data.loc[date, ma_col] < data.loc[date, next_ma_col]:
            super_stock = 0
            print(f"Date: {date}, Stock: {stock}")
            print(f"{ma_col} ({window}) is smaller than {next_ma_col} ({window_sizes[next_window_idx]})")
        ma_col = next_ma_col

  return pd.DataFrame(super_stock).transpose()

# Define window sizes for moving averages
window_sizes = [10, 20, 50, 100, 200]

# Apply the moving average calculation
data_with_ma = calculate_moving_averages(stock_list, data.copy(), window_sizes)
display(data_with_ma)

# Calculate above/below MA counts
above_below_data = calculate_above_below_ma(stock_list, data_with_ma.copy(), window_sizes)

print(above_below_data)

display(above_below_data)



def calculate_super_stock(stock_list, data, window_sizes):
  """
  Identifies super stocks for each date in the provided data.

  Args:
      stock_list (list): List of stock tickers.
      data (pandas.DataFrame): DataFrame containing stock data (Adj Close preferred).
      window_sizes (list): List of window sizes for moving averages.

  Returns:
      pandas.DataFrame: DataFrame with dates as rows and stock names as columns.
                          Each cell contains 1 (True) if the stock is a super stock for that date,
                          and 0 (False) otherwise.
  """

  # Initialize an empty DataFrame to store super stock values
  super_stock_df = pd.DataFrame(index=data.index, columns=stock_list)

  for date in data.index:
    for stock in stock_list:
      # All conditions must be met for a super stock
      is_super_stock = True

      for window in window_sizes:
        ma_col = f"{stock}_MA_{window}"
        if data.loc[date, stock] < data.loc[date, ma_col]:
          is_super_stock = False
          break  # Early exit if any condition fails

      super_stock_df.loc[date, stock] = int(is_super_stock)

  return super_stock_df
